In [ ]:
import numpy as np
import pandas as pd
from autogluon.tabular import TabularPredictor
from perpetual import PerpetualBooster
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold

In [ ]:
X_train = pd.read_csv("../../resources/santander-train.csv", index_col=0)
y_train = X_train.pop("TARGET")

In [ ]:
X_train.shape

In [ ]:
cv = KFold(shuffle=True, random_state=42)

In [ ]:
scores = []
for train, test in cv.split(X_train, y_train):
    model = PerpetualBooster(budget=1.0, objective="LogLoss")
    model.fit(X_train.iloc[train], y_train.iloc[train])
    probabilities = model.predict_proba(X_train.iloc[test])
    score = roc_auc_score(y_train.iloc[test], probabilities[:, 1])
    scores.append(score)
    print(model.number_of_trees)
print(np.mean(scores))

In [ ]:
scores = []
X_train["TARGET"] = y_train
for train, test in cv.split(X_train, y_train):
    model = TabularPredictor(label="TARGET", verbosity=0)
    model.fit(X_train.iloc[train])
    probabilities = model.predict_proba(X_train.iloc[test])
    score = roc_auc_score(y_train.iloc[test], probabilities.to_numpy()[:, 1])
    print(score)
    scores.append(score)
print(np.mean(scores))